In [47]:

import pandas as pd
from dynamic_characterization.ipcc_ar6 import characterize_co2

data = {
    "date": pd.date_range(start="2020-01-01", end="2020-12-31", freq="Y"),
    "amount": [1],
    "flow": [1],
    "activity": [None],
}

df = pd.DataFrame(data)

for index, row in df.iterrows():
    characterized_row = characterize_co2(row, period=100, cumulative=True)


# Create a new DataFrame using the date and amount arrays from characterized_row
cumulative_df = pd.DataFrame({
    "date": characterized_row.date,
    "amount": characterized_row.amount
})

cumulative_df

C:\Users\HP\AppData\Local\Temp\ipykernel_19124\2619905588.py:5: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  "date": pd.date_range(start="2020-01-01", end="2020-12-31", freq="Y"),


,date,amount
0,2020-12-31 00:00:00,0.000000e+00
1,2021-12-31 05:49:12,1.647203e-15
2,2022-12-31 11:38:24,3.193468e-15
3,2023-12-31 17:27:36,4.657213e-15
4,2024-12-30 23:16:48,6.053100e-15
...,...,...
95,2116-01-01 00:54:00,8.575179e-14
96,2116-12-31 06:43:12,8.645733e-14
97,2117-12-31 12:32:24,8.716115e-14
98,2118-12-31 18:21:36,8.786329e-14


In [53]:
import pandas as pd
from typing import List
from dynamic_characterization.ipcc_ar6 import characterize_co2, characterize_ch4
from dynamic_characterization import characterize

def compute_cumulative_rf(
    years: List[int],
    ghg: str,
    startyear: int = 2020,
    timehorizon: int = 100
) -> pd.DataFrame:
    """
    Compute cumulative radiative forcing for 1 unit GHG emissions per year.

    Parameters:
        years (List[int]): Years when emissions occur (e.g., [2020, 2025, 2030])
        ghg (str): Greenhouse gas type ("CO2", "CH4")
        startyear (int): Start year of the radiative forcing model (default is 2020)
        timehorizon (int): Duration in years for which radiative forcing is calculated (default is 100)

    Returns:
        pd.DataFrame: DataFrame with 'year' (as Timestamp) and 'cumulative_radiative_forcing'
    """

    ghg = ghg.upper()
    ghg_to_flow = {
        "CO2": 0,
        "CH4": 1
    }

    if ghg not in ghg_to_flow:
        raise ValueError(f"Unsupported GHG type: {ghg}. Supported types: {list(ghg_to_flow.keys())}")

    flow = ghg_to_flow[ghg]

    # Step 1: Create RF impulse response for 1 unit in 2020
    df = pd.DataFrame({
        "date": [pd.Timestamp(startyear)],
        "flow": [flow],
        "amount": [1],
        "activity": [None]
    })

    characterization_function_dict = {
        0: characterize_co2,
        1: characterize_ch4,
    }

    characterized_df = characterize(
        df,
        metric="radiative_forcing",
        characterization_function_dict=characterization_function_dict,
        time_horizon=timehorizon,
        time_horizon_start=pd.Timestamp(startyear)
    )

    # Step 2: Compute cumulative RF for each emission year
    results = []
    for year in years:
        years_left = startyear + timehorizon - year - 1  # max is 100 years after 2020
        rf_values = characterized_df.head(years_left)["amount"]
        cumulative_rf = rf_values.sum()

        results.append({
            "year": year,
            "cumulative_radiative_forcing": cumulative_rf
        })

    return pd.DataFrame(results)

ghg= "CH4"
cumulative_rf_df = compute_cumulative_rf(
    years=range(2020, 2030),
    ghg=ghg,
    startyear=2020,
    timehorizon=100
)
cumulative_rf_df


rf_dict = {
    (ghg, int(row["year"])): row["cumulative_radiative_forcing"]
    for _, row in cumulative_rf_df.iterrows()
}
rf_dict


{('CH4', 2020): 2.3651673669270527e-12,
 ('CH4', 2021): 2.3651198384711042e-12,
 ('CH4', 2022): 2.3650681065838066e-12,
 ('CH4', 2023): 2.36501179951239e-12,
 ('CH4', 2024): 2.3649505126261558e-12,
 ('CH4', 2025): 2.3648838055087402e-12,
 ('CH4', 2026): 2.364811198793221e-12,
 ('CH4', 2027): 2.3647321707173162e-12,
 ('CH4', 2028): 2.3646461533739266e-12,
 ('CH4', 2029): 2.364552528630073e-12}